## Train detection model

In [ ]:
%cd 1_Detection_Model

### Test run

In [ ]:
# Test run (optional)
# YOLOv5l, 1280x1280 input size, pretrained
!python train.py --project "/root/autodl-tmp/DT_SegNet/Detection_Model_Output" --img 1280 --data dtsegnet.yaml --cache --batch-size -1 --epochs 2 --weights yolov5l6.pt


### Backbone selection

#### 640x640 input size, not pretrained

In [ ]:
# YOLOv5n, 640x640 input size, not pretrained
!python train.py --project "/root/autodl-tmp/DT_SegNet/Detection_Model_Output" --img 640 --data dtsegnet.yaml --cache --batch-size -1 --epochs 9999 --patience 150 --cfg yolov5n.yaml --weights ''


In [ ]:
# YOLOv5s, 640x640 input size, not pretrained
!python train.py --project "/root/autodl-tmp/DT_SegNet/Detection_Model_Output" --img 640 --data dtsegnet.yaml --cache --batch-size -1 --epochs 9999 --patience 150 --cfg yolov5s.yaml --weights ''


In [ ]:
# YOLOv5m, 640x640 input size, not pretrained
!python train.py --project "/root/autodl-tmp/DT_SegNet/Detection_Model_Output" --img 640 --data dtsegnet.yaml --cache --batch-size -1 --epochs 9999 --patience 150 --cfg yolov5m.yaml --weights ''


In [ ]:
# YOLOv5l, 640x640 input size, not pretrained
!python train.py --project "/root/autodl-tmp/DT_SegNet/Detection_Model_Output" --img 640 --data dtsegnet.yaml --cache --batch-size -1 --epochs 9999 --patience 150 --cfg yolov5l.yaml --weights ''


#### 1280x1280 input size, not pretrained

In [ ]:
# YOLOv5n, 1280x1280 input size, not pretrained
!python train.py --project "/root/autodl-tmp/DT_SegNet/Detection_Model_Output" --img 1280 --data dtsegnet.yaml --cache --batch-size -1 --epochs 9999 --patience 150 --cfg yolov5n.yaml --weights ''


In [ ]:
# YOLOv5s, 1280x1280 input size, not pretrained
!python train.py --project "/root/autodl-tmp/DT_SegNet/Detection_Model_Output" --img 1280 --data dtsegnet.yaml --cache --batch-size -1 --epochs 9999 --patience 150 --cfg yolov5s.yaml --weights ''


In [ ]:
# YOLOv5m, 1280x1280 input size, not pretrained
!python train.py --project "/root/autodl-tmp/DT_SegNet/Detection_Model_Output" --img 1280 --data dtsegnet.yaml --cache --batch-size -1 --epochs 9999 --patience 150 --cfg yolov5m.yaml --weights ''


In [ ]:
# YOLOv5l, 1280x1280 input size, not pretrained
!python train.py --project "/root/autodl-tmp/DT_SegNet/Detection_Model_Output" --img 1280 --data dtsegnet.yaml --cache --batch-size -1 --epochs 9999 --patience 150 --cfg yolov5l.yaml --weights ''


#### 1280x1280 input size, pretrained

In [ ]:
# YOLOv5n, 1280x1280 input size, pretrained
!python train.py --project "/root/autodl-tmp/DT_SegNet/Detection_Model_Output" --img 1280 --data dtsegnet.yaml --cache --batch-size -1 --epochs 9999 --patience 150 --weights yolov5n6.pt


In [ ]:
# YOLOv5s, 1280x1280 input size, pretrained
!python train.py --project "/root/autodl-tmp/DT_SegNet/Detection_Model_Output" --img 1280 --data dtsegnet.yaml --cache --batch-size -1 --epochs 9999 --patience 150 --weights yolov5s6.pt


In [ ]:
# YOLOv5m, 1280x1280 input size, pretrained
!python train.py --project "/root/autodl-tmp/DT_SegNet/Detection_Model_Output" --img 1280 --data dtsegnet.yaml --cache --batch-size -1 --epochs 9999 --patience 150 --weights yolov5m6.pt


In [ ]:
# YOLOv5l, 1280x1280 input size, pretrained
!python train.py --project "/root/autodl-tmp/DT_SegNet/Detection_Model_Output" --img 1280 --data dtsegnet.yaml --cache --batch-size -1 --epochs 9999 --patience 150 --weights yolov5l6.pt


In [ ]:
%cd ..

## Generate dataset for segmentation model

### Import packages

In [ ]:
from pathlib import Path
from PIL import Image
from tqdm import tqdm
import pandas as pd


### Convert Detection Dataset to Segmentation Dataset

In [ ]:
# read YOLO format txt label, and perform dilation
def read_labels(label_dir, img, dilation = 1.5):
    data = pd.read_csv(str(label_dir), sep=" ", header=None,
                    names=["class", "x_center", "y_center", "width", "height"])
    img_x, img_y = img.size
    data['x0'] = ((data['x_center'] - dilation * data['width'] / 2) * img_x).astype("int")
    data['x1'] = ((data['x_center'] + dilation * data['width'] / 2) * img_x).astype("int")
    data['y0'] = ((data['y_center'] - dilation * data['height'] / 2) * img_y).astype("int")
    data['y1'] = ((data['y_center'] + dilation * data['height'] / 2) * img_y).astype("int")
    return data


In [ ]:
sub_dirs = ['train', 'test', 'val']

data_dir = Path('/root/autodl-tmp/DT_SegNet/Dataset/')
label_dir = Path('/root/autodl-tmp/DT_SegNet/Dataset/segmentation_labels')

output_root = Path('/root/autodl-tmp/DT_SegNet/Dataset/Segmentation_Dataset/')
output_data_dir = output_root / 'images'
output_label_dir = output_root / 'labels'


In [ ]:
for cset in sub_dirs:
    (output_data_dir/cset).mkdir(parents=True, exist_ok=True)
    (output_label_dir/cset).mkdir(parents=True, exist_ok=True)
    for img_path in list((data_dir/cset).glob('**/*.png')):
        print(f'Processing {str(img_path)}')

        # process cropped image
        img = Image.open(img_path)
        img = img.convert("L")
        # print(img.format, img.size, img.mode)
        labels = read_labels(img_path.with_suffix('.txt'), img)
        with tqdm(total=len(labels)) as pbar:
            for index, r in labels.iterrows():
                box = (r.x0, r.y0, r.x1, r.y1)
                region = img.crop(box)
                # region.show()
                croped_savepath = output_data_dir / cset / f'{img_path.stem}_{index}{img_path.suffix}'
                # print(croped_savepath, box)
                region.save(croped_savepath)
                pbar.update(1)
        
        # process cropped label
        img = Image.open(label_dir / f'{img_path.stem}{img_path.suffix}')
        img = img.convert("P")
        # print(img.format, img.size, img.mode)
        with tqdm(total=len(labels)) as pbar:
            for index, r in labels.iterrows():
                box = (r.x0, r.y0, r.x1, r.y1)
                region = img.crop(box)
                # region.show()
                croped_savepath = output_label_dir / cset / f'{img_path.stem}_{index}{img_path.suffix}'
                # print(croped_savepath, box)
                region.save(croped_savepath)
                pbar.update(1)


In [ ]:
for cset in sub_dirs:
    datas = pd.DataFrame({
        'data': [f'images/{cset}/{x.name}' for x in sorted((output_data_dir/cset).glob('*.png'))],
        'label': [f'labels/{cset}/{x.name}' for x in sorted((output_label_dir/cset).glob('*.png'))],
    })

    datas.to_csv(output_root / f'{cset}.txt', header=None, index=None, sep=' ')


### Convert Detection Dataset to Segmentation Dataset (KFold)

In [ ]:
kf_ids = [0, 1, 2, 3, 4]
sub_dirs = ['train', 'val']


In [ ]:
for kf_id in kf_ids:

    data_dir = Path(f'/root/autodl-tmp/DT_SegNet/Dataset/KFold/{kf_id}')
    label_dir = Path(f'/root/autodl-tmp/DT_SegNet/Dataset/segmentation_labels')

    output_root = Path(f'/root/autodl-tmp/DT_SegNet/Dataset/SD_KF/{kf_id}')
    output_data_dir = output_root / 'images'
    output_label_dir = output_root / 'labels'
    
    for cset in sub_dirs:
        (output_data_dir/cset).mkdir(parents=True, exist_ok=True)
        (output_label_dir/cset).mkdir(parents=True, exist_ok=True)
        for img_path in list((data_dir/cset).glob('**/*.png')):
            print(f'Processing {str(img_path)}')

            # process cropped image
            img = Image.open(img_path)
            img = img.convert("L")
            # print(img.format, img.size, img.mode)
            labels = read_labels(img_path.with_suffix('.txt'), img)
            with tqdm(total=len(labels)) as pbar:
                for index, r in labels.iterrows():
                    box = (r.x0, r.y0, r.x1, r.y1)
                    region = img.crop(box)
                    # region.show()
                    croped_savepath = output_data_dir / cset / f'{img_path.stem}_{index}{img_path.suffix}'
                    # print(croped_savepath, box)
                    region.save(croped_savepath)
                    pbar.update(1)

            # process cropped label
            img = Image.open(label_dir / f'{img_path.stem}{img_path.suffix}')
            img = img.convert("P")
            # print(img.format, img.size, img.mode)
            with tqdm(total=len(labels)) as pbar:
                for index, r in labels.iterrows():
                    box = (r.x0, r.y0, r.x1, r.y1)
                    region = img.crop(box)
                    # region.show()
                    croped_savepath = output_label_dir / cset / f'{img_path.stem}_{index}{img_path.suffix}'
                    # print(croped_savepath, box)
                    region.save(croped_savepath)
                    pbar.update(1)
                
    for cset in sub_dirs:
        datas = pd.DataFrame({
            'data': [f'images/{cset}/{x.name}' for x in sorted((output_data_dir/cset).glob('*.png'))],
            'label': [f'labels/{cset}/{x.name}' for x in sorted((output_label_dir/cset).glob('*.png'))],
        })

        datas.to_csv(output_root / f'{cset}.txt', header=None, index=None, sep=' ')

## Train segmentation model

In [ ]:
Path('/root/autodl-tmp/DT_SegNet/Segmentation_Model_Output').mkdir(parents=True, exist_ok=True)
Path('/root/autodl-tmp/DT_SegNet/Baseline_Model_Output').mkdir(parents=True, exist_ok=True)
%cd 3_Segmentation_Model

### Test run

In [ ]:
!python train.py --iters 200 --config configs/dtsegnet/segformer_b1.yml --do_eval --use_vdl --save_interval 100 --keep_checkpoint_max 1 --save_dir "/root/autodl-tmp/DT_SegNet/Segmentation_Model_Output/B1" > "/root/autodl-tmp/DT_SegNet/Segmentation_Model_Output/B1.log"


### Backbone selection

In [ ]:
# SegFormer B0
!python train.py --config configs/dtsegnet/segformer_b0.yml --do_eval --use_vdl --save_interval 200 --save_dir "/root/autodl-tmp/DT_SegNet/Segmentation_Model_Output/B0" > "/root/autodl-tmp/DT_SegNet/Segmentation_Model_Output/B0.log"


In [ ]:
# SegFormer B1
!python train.py --config configs/dtsegnet/segformer_b1.yml --do_eval --use_vdl --save_interval 200 --save_dir "/root/autodl-tmp/DT_SegNet/Segmentation_Model_Output/B1" > "/root/autodl-tmp/DT_SegNet/Segmentation_Model_Output/B1.log"


### Train baselines

In [ ]:
# SegFormer B0
!python train.py --config configs/baseline/segformer_b0.yml --do_eval --use_vdl --save_interval 100  --keep_checkpoint_max 1 --save_dir "/root/autodl-tmp/DT_SegNet/Baseline_Model_Output/segformer_b0" > "/root/autodl-tmp/DT_SegNet/Baseline_Model_Output/segformer_b0.log"


In [ ]:
# SegFormer B1
!python train.py --config configs/baseline/segformer_b1.yml --do_eval --use_vdl --save_interval 100  --keep_checkpoint_max 1 --save_dir "/root/autodl-tmp/DT_SegNet/Baseline_Model_Output/segformer_b1" > "/root/autodl-tmp/DT_SegNet/Baseline_Model_Output/segformer_b1.log"


In [ ]:
# U-Net
!python train.py --config configs/baseline/unet.yml --do_eval --use_vdl --save_interval 100 --keep_checkpoint_max 1 --save_dir "/root/autodl-tmp/DT_SegNet/Baseline_Model_Output/unet" > "/root/autodl-tmp/DT_SegNet/Baseline_Model_Output/unet.log"


In [ ]:
# UNet 3+
!python train.py --config configs/baseline/unet_3plus.yml --do_eval --use_vdl --save_interval 100  --keep_checkpoint_max 1 --save_dir "/root/autodl-tmp/DT_SegNet/Baseline_Model_Output/unet_3plus" > "/root/autodl-tmp/DT_SegNet/Baseline_Model_Output/unet_3plus.log"


In [ ]:
# DeepLabV3P
!python train.py --config configs/baseline/deeplabv3p_resnet50.yml --do_eval --use_vdl --save_interval 100  --keep_checkpoint_max 1 --save_dir "/root/autodl-tmp/DT_SegNet/Baseline_Model_Output/deeplabv3p_resnet50" > "/root/autodl-tmp/DT_SegNet/Baseline_Model_Output/deeplabv3p_resnet50.log"
